In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

def spacyCleanText(summary):
    """
    Works well. Final function. Lemmatizes words with some basic cleaning.
    """
#     stops = STOP_WORDS
    cleanSum = re.sub('[^a-zA-Z]',' ', summary) # wanna keep all letters
    cleanSum = cleanSum.lower()                 # write all characters in lowercase
#     nlp = spacy.load('en')
#     doc = nlp(cleanSum)
#     cleanSum = ' '.join([token.lemma_ for token in doc if not token in set(stops)])
    return (cleanSum)

In [3]:
# dataset paths
aspect_Reddit_path = './Papers/Aspect_Argen/requestitem_export_9dda3a7b-56c5-4087-a995-243924251b82/reddit_training_data'
aspect_Crawled_path = './Papers/Aspect_Argen/requestitem_export_9dda3a7b-56c5-4087-a995-243924251b82/cc_training_data'
aspect_training_path = './Papers/Aspect_Argen/requestitem_export_9dda3a7b-56c5-4087-a995-243924251b82/argument_aspect_detection_v1.0/cross_topic'
arguana_path = './Argument_Generation/args.csv'

# Arguana Corpus

In [4]:
arguana = pd.read_csv(arguana_path)

In [5]:
arguana['Arguments'] = [spacyCleanText(arg) for arg in arguana['Arguments']]
arguana['Counters'] = [spacyCleanText(arg) for arg in arguana['Counters']]
arguana['Characters_Args'] = [len(arg) for arg in arguana['Arguments']]
wordList = [arg.split() for arg in arguana['Arguments']]
arguana['NumWords_Args'] = [len(x) for x in wordList]

In [6]:
arguana.describe()

,Characters_Args,NumWords_Args
count,8024.000,8024.000
mean,1304.014,206.035
std,640.391,99.766
min,245.000,41.000
25%,884.000,140.000
50%,1165.500,185.000
75%,1554.500,247.000
max,5375.000,884.000


In [10]:
# arguana.loc[arguana['NumWords_Args'] == 884]['Arguments'] 
# 1551, 1554, 5267, 5270
arguana['Arguments'][1551]

'the mandate falls under taxation and general welfare powers  an insurance mandate would be enforced through income tax laws  so even if a simple mandate were not a valid  regulation   it still could fall easily within congress s plenary power to tax income  for instance  anyone purchasing insurance could be given an income tax credit  and those not purchasing could be assessed an income tax penalty   the only possible constitutional restriction is an archaic provision saying that if congress imposes anything that amounts to a  head tax  or  poll tax   that is  taxing people simply as people rather than taxing their income   then it must do so uniformly  that is  the same amount per person   this technical restriction is easily avoided by using income tax laws   purists complain that taxes should be proportional to actual income and should not be used mainly to regulate economic behaviour  but our tax code  for better or worse  is riddled with such regulatory provisions and so they are

NOTE: Arguana contains dublicates and so 8024 isn't the correct length. More like ~5000.

# Aspect Controlled Generation Paper: Reddit data (finetuning)

In [11]:
import jsonlines
import os

reddit_Args = []
for subdir, dirs, files in os.walk(aspect_Reddit_path):
    for file in files:
        path = os.path.join(subdir, file)
        if path[-1] is 'l':
            with jsonlines.open(path) as f:
                for line in f.iter():
                    reddit_Args.append(line['sent'])


In [17]:
reddit = pd.DataFrame()
reddit['Comments']=reddit_Args
# reddit['Comments'] = [spacyCleanText(arg) for arg in reddit['Comments']]
reddit['Characters'] = [len(arg) for arg in reddit['Comments']]
wordList = [arg.split() for arg in reddit['Comments']]
reddit['NumWords_Args'] = [len(x) for x in wordList]

In [18]:
reddit.describe()

,Characters,NumWords_Args
count,3637603.000,3637603.000
mean,127.139,24.695
std,73.083,13.787
min,7.000,1.000
25%,77.000,15.000
50%,113.000,22.000
75%,161.000,31.000
max,3646.000,187.000


In [24]:
reddit.loc[reddit['NumWords_Args'] == 187]['Comments']

2892664    And also I know someone that has a maid that h...
Name: Comments, dtype: object

In [25]:
reddit['Comments'][2892664]

'And also I know someone that has a maid that her mother died , he told her to go visit her family for at least 3 months and she refused her self saying that her mother just died what would be going to thier do anything she isn ’ t coming from death just because I came there 3-that ’ s not even a thing I think you are talking about the “ punishment of having sex before marriage or after marriage with another women ” while this thing exists the punishment isn ’ t for women only and there have to be 5 Witnesses other then you and the women for this punishment to be a thing and it rarely happens and even if for what ever reason you were able to get 5 Witnesses they will most likley let you go , death isn ’ t an easy thing to do and god has mercy , I deal with idiotic people like you almost everyday they hear rumors and think it ’ s true , you aren ’ t even better then North Korean people'

# Aspect Controlled Generation Paper: Crawled data (finetuning)

Done only for first 3637603 arguments due to large size. 

In [17]:
crawled_Args = []
for subdir, dirs, files in os.walk(aspect_Crawled_path):
    for file in files:
        path = os.path.join(subdir, file)
        if path[-1] is 'l':
            with jsonlines.open(path) as f:
                for line in f.iter():
                    crawled_Args.append(line['sent'])
len(crawled_Args)

12602343

In [19]:
crawled = pd.DataFrame()
crawled['Arguments']=crawled_Args[0:3637603]
crawled['Arguments'] = [spacyCleanText(arg) for arg in crawled['Arguments']]
crawled['Characters'] = [len(arg) for arg in crawled['Arguments']]
wordList = [arg.split() for arg in crawled['Arguments']]
crawled['NumWords_Args'] = [len(x) for x in wordList]

In [20]:
crawled.describe()

,Characters,NumWords_Args
count,3637603.000,3637603.000
mean,193.434,30.071
std,95.965,14.410
min,10.000,1.000
25%,128.000,20.000
50%,177.000,28.000
75%,239.000,37.000
max,1161.000,171.000


# Aspect Controlled Generation Paper: Training data 

In [23]:
training_Args = []
for subdir, dirs, files in os.walk(aspect_training_path):
    for file in files:
        path = os.path.join(subdir, file)
        if path[-1] is 'l':
            with jsonlines.open(path) as f:
                for line in f.iter():
                    training_Args.append(line['sentence'])


5032

In [25]:
training = pd.DataFrame()
training['Arguments']=training_Args
training['Arguments'] = [spacyCleanText(arg) for arg in training['Arguments']]
training['Characters'] = [len(arg) for arg in training['Arguments']]
wordList = [arg.split() for arg in training['Arguments']]
training['NumWords_Args'] = [len(x) for x in wordList]

In [26]:
training.describe()

,Characters,NumWords_Args
count,5032.000,5032.000
mean,141.432,23.099
std,67.789,10.920
min,18.000,2.000
25%,90.000,15.000
50%,130.000,22.000
75%,180.000,29.000
max,448.000,66.000
